In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions

pipeline_options_dict = {
    'runner': 'DirectRunner',  # Ou 'DataflowRunner' se você estiver executando no Dataflow
    'streaming': True,
}

pipeline_options = PipelineOptions.from_dictionary(pipeline_options_dict)

with beam.Pipeline(options = pipeline_options) as pipeline:
    processing = (
        pipeline
        | beam.io.ReadFromPubSub(subscription='projects/playground-s-11-a3b55282/subscriptions/dataflow').with_output_types(bytes)
        | beam.Map(print)#beam.WindowInto(beam.window.FixedWindows(window_size), trigger=beam.trigger.AfterWatermark(), accumulation_mode=beam.trigger.AccumulationMode.DISCARDING)
        #| beam.io.WriteToText('./test/')
    )


In [ ]:
from apache_beam.coders.coders import TupleCoder, FloatCoder, ListCoder
from apache_beam.transforms.userstate import BagStateSpec
import apache_beam as beam
from sys import argv
import argparse
from apache_beam.options.pipeline_options import PipelineOptions

parser = argparse.ArgumentParser()

#pipeline_options = PipelineOptions.from_dictionary(pipeline_options_dict)

def to_json(data):
   import json
   data = json.loads(data)
   return data[0]
   
def streaming_columns(crypto_dict):
   return dict(code=crypto_dict.get('code'), rate=crypto_dict.get('rate'), volume=crypto_dict.get('volume'), cap=crypto_dict.get('cap'), circulatingSupply=crypto_dict.get('circulatingSupply'), totalSupply=crypto_dict.get('totalSupply'), maxSupply=crypto_dict.get('maxSupply'))

class SetCoinKey(beam.DoFn):
   def process(self, element):
      yield element['code'], element


class MinMaxBitcoinPriceFn(beam.DoFn):

   PRICE_STATE = BagStateSpec('price_state', TupleCoder((FloatCoder(), FloatCoder())))

   def process(self, element, prev_state = beam.DoFn.StateParam(PRICE_STATE)):
      current_price = element[1]['rate']
      previous_prices = list(prev_state.read())

      if previous_prices:

         if current_price>previous_prices[0][0]:
            prev_state.clear()

            prev_state.add((current_price, previous_prices[0][1]))
            
         elif current_price<previous_prices[0][1]:
            prev_state.clear()
            prev_state.add((previous_prices[0][0], current_price))

      else:
         prev_state.add((current_price, current_price))

      actual_prices = list(prev_state.read()) 
      element[1]['max_price'] = actual_prices[0][0]
      element[1]['min_price'] = actual_prices[0][1]

      yield element

def format_for_bigquery(element):
    currency, data = element
    return {
        'code': data['code'],
        'rate': data['rate'],
        'volume': data['volume'],
        'cap': data['cap'],
        'circulatingSupply': data['circulatingSupply'],
        'totalSupply': data['totalSupply'],
        'maxSupply': data['maxSupply'],
        'max_price': data['max_price'],
        'min_price': data['min_price'],
    }
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    known_args = parser.parse_known_args()

    p = beam.Pipeline(options=PipelineOptions())
    (
    p
    | beam.io.ReadFromPubSub(subscription='projects/playground-s-11-a7a73cdc/subscriptions/dataflow').with_output_types(bytes) 
    | beam.Map(lambda x: x.decode('utf-8')) 
    | beam.Map(to_json)
    | beam.Map(streaming_columns)
    | beam.ParDo(SetCoinKey())
    | beam.ParDo(MinMaxBitcoinPriceFn())
    | beam.Map(format_for_bigquery)
    | beam.io.WriteToBigQuery('playground-s-11-a7a73cdc:samples.weather_stations',
    create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
    write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
    schema='code:STRING,rate:FLOAT,volume:INTEGER,cap:INTEGER,circulatingSupply:INTEGER,totalSupply:INTEGER,maxSupply:INTEGER,max_price:FLOAT,min_price:FLOAT')
    #beam.WindowInto(beam.window.FixedWindows(window_size), trigger=beam.trigger.AfterWatermark(), accumulation_mode=beam.trigger.AccumulationMode.DISCARDING)
    )
    result = p.run()
    result.wait_until_finish()




   